In [ ]:
from pathlib import Path
from pprint import pprint

import torch

import lerobot
from lerobot.common.datasets.lerobot_dataset import (
    LeRobotDataset,
    LeRobotDatasetMetadata,
)

root_path = Path("/Users/pkm/Projects/bottender/datasets/example")

dataset = LeRobotDataset(
    root=root_path,
)

# And see how many frames you have:
print(f"Selected episodes: {dataset.episodes}")
print(f"Number of episodes selected: {dataset.num_episodes}")
print(f"Number of frames selected: {dataset.num_frames}")

Generating train split: 451 examples [00:00, 1137.53 examples/s]


ValueError: One or several timestamps unexpectedly violate the tolerance inside episode range.
                This might be due to synchronization issues during data collection.
                
[{'diff': np.float32(0.09967995),
  'episode_index': 0,
  'timestamps': [np.float32(0.0), np.float32(0.09967995)]},
 {'diff': np.float32(0.09969902),
  'episode_index': 0,
  'timestamps': [np.float32(0.09967995), np.float32(0.19937897)]},
 {'diff': np.float32(0.10063791),
  'episode_index': 0,
  'timestamps': [np.float32(0.19937897), np.float32(0.30001688)]},
 {'diff': np.float32(0.10389304),
  'episode_index': 0,
  'timestamps': [np.float32(0.30001688), np.float32(0.40390992)]},
 {'diff': np.float32(0.09781909),
  'episode_index': 0,
  'timestamps': [np.float32(0.40390992), np.float32(0.501729)]},
 {'diff': np.float32(0.09910989),
  'episode_index': 0,
  'timestamps': [np.float32(0.501729), np.float32(0.6008389)]},
 {'diff': np.float32(0.09958196),
  'episode_index': 0,
  'timestamps': [np.fl